In [11]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier
from numpy import loadtxt
import numpy as np
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score

In [3]:
df1 = pd.read_csv("../data/one-hotted-tedsd/updated_2016_3_v2.csv")
print("Finished processing ... df1");
df2 = pd.read_csv("../data/one-hotted-tedsd/updated_2016_2_v2.csv")
print("Finished processing ... df2");
df3 = pd.read_csv("../data/one-hotted-tedsd/update_2016_1_v3.csv")
print("Finished processing ... df3");
frames = [df1, df2, df3]
print("Finished processing ... frames");
original_df = pd.concat(frames)
print("This is the very original dataset size: " + str(original_df.shape))

df_copy = original_df.copy(deep=True).sample(frac=1)
dataset = df_copy.to_numpy()

Finished processing ... df1
Finished processing ... df2
Finished processing ... df3
Finished processing ... frames


/Users/jonahwu/miniconda3/envs/pysal2_1_release/lib/python3.6/site-packages/ipykernel_launcher.py:9: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  if __name__ == '__main__':


This is the very original dataset size: (1458045, 248)


In [4]:
#Extract Features
dataFields = {}
for x in range(len(df_copy.columns)):
	dataFields.update({df_copy.columns[x]: x})
print(dataFields)
indx = dataFields["reason"]
Y = dataset[:,[indx]]
Y = np.select([(Y == 1)], [1] , default = 0) #Set to binary classification

featureWeights = pd.read_csv("../data/rankedFeatures.csv")
fields = list(featureWeights["field"])
desiredChars = [fields[x] for x in range(143)]
print(desiredChars)
indxsToDelete = []
for key, value in dataFields.items():
    if key not in desiredChars:
        indxsToDelete.append(value)

X = np.delete(dataset, indxsToDelete, 1)
print("Shape of dataset following feature deletion: ", X.shape)
print("Label Dimensions:", Y.shape)


{'ALCFLG': 0, 'AMPHFLG': 1, 'ARRESTS': 2, 'BARBFLG': 3, 'BENZFLG': 4, 'COKEFLG': 5, 'DSMCRIT': 6, 'HALLFLG': 7, 'HERFLG': 8, 'IDU': 9, 'INHFLG': 10, 'MARFLG': 11, 'METHFLG': 12, 'MTHAMFLG': 13, 'NOPRIOR': 14, 'OPSYNFLG': 15, 'OTCFLG': 16, 'OTHERFLG': 17, 'PCPFLG': 18, 'PSYPROB': 19, 'SEDHPFLG': 20, 'STIMFLG': 21, 'TRNQFLG': 22, 'age_approx': 23, 'alcdrug_alc': 24, 'alcdrug_both': 25, 'alcdrug_drug': 26, 'alcdrug_none': 27, 'daywait_approx': 28, 'division_enc': 29, 'division_esc': 30, 'division_ma': 31, 'division_mt': 32, 'division_ne': 33, 'division_pc': 34, 'division_sa': 35, 'division_territory': 36, 'division_wnc': 37, 'division_wsc': 38, 'educ_approx': 39, 'employ_full': 40, 'employ_nlf': 41, 'employ_part': 42, 'employ_unemployed': 43, 'employ_unknown': 44, 'ethnic_cuban': 45, 'ethnic_latino': 46, 'ethnic_mexican': 47, 'ethnic_none': 48, 'ethnic_puerto': 49, 'ethnic_unknown': 50, 'firstuse1_approx': 51, 'firstuse2_approx': 52, 'freq1_daily': 53, 'freq1_none': 54, 'freq1_some': 55, 

In [5]:
test_size = .05
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=test_size)
eval_set=[(X_train, y_train.ravel()), (X_test, y_test.ravel())]

In [9]:
model = RandomForestClassifier(max_depth=6, verbose=1, n_estimators=150)
model.fit(X_train, y_train.ravel())
print(model)

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=6, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=150,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=1, warm_start=False)


[Parallel(n_jobs=1)]: Done 150 out of 150 | elapsed: 15.9min finished


In [12]:
y_pred = model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy: %.2f%%" % (accuracy*100.0))
precision = precision_score(y_test, y_pred)
print('Precision: %f' % precision)
# recall: tp / (tp + fn)
recall = recall_score(y_test, y_pred)
print('Recall: %f' % recall)
# f1: 2 tp / (2 tp + fp + fn)
f1 = f1_score(y_test, y_pred)
print('F1 score: %f' % f1)

print("Train scores...")
y_pred = model.predict(X_train)
accuracy = accuracy_score(y_train, y_pred)
print("Accuracy: %.2f%%" % (accuracy*100.0))
precision = precision_score(y_train, y_pred)
print('Precision: %f' % precision)
# recall: tp / (tp + fn)
recall = recall_score(y_train, y_pred)
print('Recall: %f' % recall)
# f1: 2 tp / (2 tp + fp + fn)
f1 = f1_score(y_train, y_pred)
print('F1 score: %f' % f1)

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 150 out of 150 | elapsed:    0.7s finished


Accuracy: 65.26%
Precision: 0.789805
Recall: 0.213115
F1 score: 0.335659
Train scores...


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 150 out of 150 | elapsed:   14.2s finished


Accuracy: 65.17%
Precision: 0.785985
Recall: 0.209670
F1 score: 0.331034
